# Gallery Example: M/M/1-PS Multiclass Queue

This example demonstrates an M/M/1 queue with Processor Sharing and multiple classes:
- **Arrivals**: Exponential (separate Poisson process per class)
- **Service**: Exponential (class-dependent)
- **Servers**: 1 server
- **Scheduling**: Processor Sharing (PS)
- **Classes**: 2 job classes

In [ ]:
from line_solver import *
GlobalConstants.set_verbose(VerboseLevel.STD)

In [ ]:
def gallery_mm1_ps_multiclass():
    """Create M[2]/M[2]/1-PS multiclass queueing model"""
    model = Network('M[2]/M[2]/1-PS')
    
    # Block 1: nodes
    source = Source(model, 'mySource')
    queue = Queue(model, 'myQueue', SchedStrategy.PS)  # Processor Sharing
    sink = Sink(model, 'mySink')
    
    # Block 2: classes
    oclass1 = OpenClass(model, 'myClass1')
    source.set_arrival(oclass1, Exp(1))
    queue.set_service(oclass1, Exp(4))
    
    oclass2 = OpenClass(model, 'myClass2')
    source.set_arrival(oclass2, Exp(0.5))
    queue.set_service(oclass2, Exp(4))
    
    # Block 3: topology
    P = model.init_routing_matrix()
    P.add_route(oclass1, source, queue, 1.0)
    P.add_route(oclass1, queue, sink, 1.0)
    P.add_route(oclass2, source, queue, 1.0)
    P.add_route(oclass2, queue, sink, 1.0)
    model.link(P)
    
    return model

# Create the model
model = gallery_mm1_ps_multiclass()

## About Multiclass Processor Sharing

Under Processor Sharing:
- All jobs in the system share the server capacity equally
- Each of n jobs receives 1/n of capacity, regardless of class
- Response time is proportional to service requirement

**Class 1**: λ₁ = 1, μ₁ = 4 → ρ₁ = 0.25
**Class 2**: λ₂ = 0.5, μ₂ = 4 → ρ₂ = 0.125
**Total utilization**: ρ = 0.375

PS scheduling provides fairness across classes based on service requirements.

In [ ]:
# Solve with multiple solvers
print("\n=== Solver Results ===")

# MVA Solver
solver_mva = MVA(model)
avg_table_mva = solver_mva.avg_table()
print("\nMVA Solver:")
print(avg_table_mva)

# CTMC Solver
solver_ctmc = CTMC(model, cutoff=15)
avg_table_ctmc = solver_ctmc.avg_table()
print("\nCTMC Solver:")
print(avg_table_ctmc)

# Fluid Solver
solver_fluid = FLD(model)
avg_table_fluid = solver_fluid.avg_table()
print("\nFluid Solver:")
print(avg_table_fluid)